In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('/Users/lyeajin/Downloads/playground-series-s4e7/보험이진분류train.csv')

In [3]:
test = pd.read_csv('/Users/lyeajin/Downloads/playground-series-s4e7/test보험이진분류.csv')

In [4]:
df = pd.read_csv("/Users/lyeajin/Downloads/train-3.csv")

In [5]:
train = pd.concat([train, df], ignore_index=True)

In [6]:
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

In [7]:
train["car_sum"] = train["Vehicle_Damage"].astype("str") + train["Driving_License"].astype("str") + train["Previously_Insured"].astype("str")
train["car_sum"] = train["car_sum"].astype("category").cat.codes

test["car_sum"] = test["Vehicle_Damage"].astype("str") + test["Driving_License"].astype("str") + test["Previously_Insured"].astype("str")
test["car_sum"] = test["car_sum"].astype("category").cat.codes

In [8]:
gender_dummies = pd.get_dummies(train['Gender'], prefix='Gender')
train = pd.concat([train, gender_dummies], axis=1)

vehicle_age_dummies = pd.get_dummies(train['Vehicle_Age'], prefix='Vehicle_Age')
train = pd.concat([train, vehicle_age_dummies], axis=1)

vehicle_damage_dummies = pd.get_dummies(train['Vehicle_Damage'], prefix='Vehicle_Damage')
train = pd.concat([train, vehicle_damage_dummies], axis=1)

In [9]:
gender_dummies_test = pd.get_dummies(test['Gender'], prefix='Gender')
test = pd.concat([test, gender_dummies_test], axis=1)

vehicle_age_dummies_test = pd.get_dummies(test['Vehicle_Age'], prefix='Vehicle_Age')
test = pd.concat([test, vehicle_age_dummies_test], axis=1)

vehicle_damage_dummies_test = pd.get_dummies(test['Vehicle_Damage'], prefix='Vehicle_Damage')
test = pd.concat([test, vehicle_damage_dummies_test], axis=1)

In [10]:
#열 이름 변경 
train.rename(columns={
    'Vehicle_Age_1-2 Year': 'Vehicle_Age_1_2_Year',
    'Vehicle_Age_< 1 Year': 'Vehicle_Age_lt_1_Year',
    'Vehicle_Age_> 2 Years': 'Vehicle_Age_gt_2_Years'
}, inplace=True)

In [11]:
test.rename(columns={
    'Vehicle_Age_1-2 Year': 'Vehicle_Age_1_2_Year',
    'Vehicle_Age_< 1 Year': 'Vehicle_Age_lt_1_Year',
    'Vehicle_Age_> 2 Years': 'Vehicle_Age_gt_2_Years'
}, inplace=True)

In [12]:
from sklearn.preprocessing import MinMaxScaler

selected_columns = ['Annual_Premium', 'Policy_Sales_Channel', 'Vintage']
minMax = MinMaxScaler()

# 훈련 데이터와 테스트 데이터 스케일링
train[selected_columns] = minMax.fit_transform(train[selected_columns])
test[selected_columns] = minMax.transform(test[selected_columns])

In [ ]:
pip install hyperopt

In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK

# 데이터 전처리 및 분리
X = train.drop(['Response', 'Gender', 'Vehicle_Age', 'Vehicle_Damage'], axis=1)  # 특징 열 선택
y = train['Response']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
# 하이퍼파라미터 공간 정의
param_space = {
    'iterations': hp.quniform('iterations', 100, 1000, 100),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'depth': hp.quniform('depth', 4, 10, 1),
    'l2_leaf_reg': hp.loguniform('l2_leaf_reg', np.log(1e-3), np.log(10)),
    'bagging_temperature': hp.uniform('bagging_temperature', 0, 1)
}

In [17]:
# Stratified K-Fold 교차 검증 설정
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# 목표 함수 정의
def objective(params):
    clf = CatBoostClassifier(
        iterations=int(params['iterations']),
        learning_rate=params['learning_rate'],
        depth=int(params['depth']),
        loss_function='Logloss',
        l2_leaf_reg=params['l2_leaf_reg'],
        bagging_temperature=params['bagging_temperature'],
        leaf_estimation_iterations=10,
        logging_level='Silent',
        random_seed=42
    )
    
    auc_scores = []
    for train_idx, valid_idx in skf.split(X_train, y_train):
        print("Training fold...")
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]
        
        train_pool = Pool(X_tr, y_tr)
        valid_pool = Pool(X_val, y_val)
        
        clf.fit(train_pool, eval_set=valid_pool, use_best_model=True)
        
        y_val_pred = clf.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, y_val_pred)
        auc_scores.append(auc)
    
    mean_auc = np.mean(auc_scores)
    print(f"Mean AUC: {mean_auc}")
    return {'loss': -mean_auc, 'status': STATUS_OK}

In [18]:
# Trials 객체 생성
trials = Trials()

# 베이지안 최적화 수행
best_params = fmin(fn=objective,
                   space=param_space,
                   algo=tpe.suggest,
                   max_evals=25,
                   trials=trials,
                   rstate=np.random.default_rng(42))

# 최적 하이퍼파라미터 출력
print("Best Parameters:", best_params)

Training fold...                                                                
Training fold...                                                                
Training fold...                                                                
Mean AUC: 0.8794752322337955                                                    
Training fold...                                                                
Training fold...                                                                
Training fold...                                                                
Mean AUC: 0.8753220087901806                                                    
Training fold...                                                                
Training fold...                                                                
Training fold...                                                                
Mean AUC: 0.8727898191400257                                                    
Training fold...            

In [19]:
# 최적의 하이퍼파라미터를 사용하여 최종 모델 훈련
model = CatBoostClassifier(
    iterations=best_params['iterations'],
    learning_rate=best_params['learning_rate'],
    loss_function='Logloss',
    depth=best_params['depth'],
    l2_leaf_reg=best_params['l2_leaf_reg'],
    bagging_temperature=best_params['bagging_temperature'],
    eval_metric='AUC',
    leaf_estimation_iterations=10,
    use_best_model=True,
    logging_level='Silent',
    random_seed=42
)

# train_pool 및 eval_set 정의 (eval_set는 검증 데이터를 의미)
train_pool = Pool(X_train, y_train)
eval_pool = Pool(X_test, y_test)

In [20]:
# 모델 훈련
model.fit(train_pool, eval_set=eval_pool)

In [21]:
# 테스트 세트 예측 및 평가
y_pred_proba = model.predict_proba(X_test)[:, 1]  # 양성 클래스의 확률을 사용
roc_auc = roc_auc_score(y_test, y_pred_proba)
print("ROC AUC Score:", roc_auc)

ROC AUC Score: 0.8796906865419061


In [ ]:
submission = pd.read_csv('')